# Uma análise quantitativa dos hotéis em Recife a partir dos dados do Trivago

Ângelo de Sant'Ana &mdash;
assd {em} cin.ufpe.br;

Antônio Rodrigues &mdash;
acpr {em} cin.ufpe.br;

Juliana Carvalho &mdash;
jco2 {em} cin.ufpe.br

junho de 2017

In [11]:
from bs4 import BeautifulSoup
import re

##### Localização dos dados relevantes

Abaixo começamos a localização dos elementos importantes das páginas .html para reconhecimento dos conjuntos que contêm
os valores que desejamos usar na nossa análise.

In [93]:
# fazer um iterador para capturar todas as páginas de 1 a 5

urlAvaliacoes = "pages/avaliacoesPag1.html"

parsed_page_avaliacoes = BeautifulSoup(open(urlAvaliacoes),"html.parser")

# para o nome do hotel
# class_="name__copytext m-0 item__slideout-toggle"

listaHoteis = []

nomesDosHoteisAvaliacoes = parsed_page_avaliacoes.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

divsAvaliacoes = parsed_page_avaliacoes.find_all('div', class_="rat-chart__group rat-chart__group--collapsed")

avaliacaoToInt = lambda x: int(re.sub('[\/][0-9]{3}', '', re.sub('[a-zA-Z\s]+','',x)))

for temp in divsAvaliacoes:
    valoresAvaliacoes = temp.find_all(class_="rat-chart__number");
    
#
#
# ficar atento aos casos em que não há avaliação
#
#

    for valor in valoresAvaliacoes:
        if "Não" in valor.text:
            # colocar '0' na posição
            print(valor.text)
        else:
            print(int(re.sub('[\/][0-9]{3}', '', re.sub('[a-zA-ZzáàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ\s]+','', valor.text))))

    print("\n")

# print(nomesDosHoteisAvaliacoes)

# for hotel in nomesDosHoteisAvaliacoes:
#     listaHoteis.append(dict(zip(["nome_do_hotel", "localizacao", "quartos", "servico", "limpeza", "custo-beneficio", \
#                                  "conforto", "facilidades", "predio", "cafe_da_manha", "comida"], \
#                                 [hotel.text, , , , , , , , , , ])))

# print(divsAvaliacoes)

# para os conjuntos das avaliações
# class_="rat-chart__group rat-chart__group--collapsed"

# para o item de avaliação exibido
# class_="rat-chart__item"

# para o nome da característica (coluna)
# class_="rat-chart__label"

# para o valor da avaliação
# class_="parentesis-wrapper"


92
84
79
85
78
85
83
80
Não avaliadoNão disponível
76


89
80
84
81
76
81
80
75
Não avaliadoNão disponível
77


86
82
79
84
77
83
83
79
70
75


84
77
82
86
77
82
82
82
75
75


89
84
88
88
79
89
85
85
Não avaliadoNão disponível
Não avaliadoNão disponível


Não avaliadoNão disponível
Não avaliadoNão disponível
83
87
Não avaliadoNão disponível
83
Não avaliadoNão disponível
81
Não avaliadoNão disponível
Não avaliadoNão disponível


91
81
84
85
77
84
81
78
78
76


88
83
84
87
79
86
84
81
72
80


84
72
78
77
75
72
78
70
Não avaliadoNão disponível
78


89
76
75
81
75
77
79
72
78
80


82
85
82
88
82
87
82
85
Não avaliadoNão disponível
76


89
74
81
79
74
78
76
77
74
76


87
76
85
82
81
78
77
Não avaliadoNão disponível
Não avaliadoNão disponível
Não avaliadoNão disponível


82
62
70
66
67
64
64
62
77
74


85
75
81
79
75
75
75
72
76
80


89
76
78
74
73
73
71
75
Não avaliadoNão disponível
Não avaliadoNão disponível


84
67
76
73
70
71
72
71
76
Não avaliadoNão disponível


85
85
87
90
82
89
85
88


In [ ]:
urlInfos = "pages/InfoPag1.html"

parsed_page_infos = BeautifulSoup(open(urlInfos,encoding="latin1"),"html.parser")

nomesDosHoteisInfos = parsed_page_infos.find_all('h3', class_="name__copytext m-0 item__slideout-toggle")

# para todas as comodidades do hotel
# class_="sl-box__block clearfix"

divsInfos = parsed_page_infos.find_all('div', class_="sl-box__block clearfix")

# print(divsInfos)

# para cada uma das comodidades do hotel
# class_="all-amenities__group mb-gutter"

# sobre os quartos
# class_="sl-box__content"